In [64]:
import datetime
import logging
 
from google.cloud import bigquery

 
projects = [
    'bigquery-public-data',
]

In [65]:
import os

In [66]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/krishna/Downloads/My Project 11220-3961afce01cc.json"

In [98]:
def get_bq_data():
    """Gets metadata from public BigQuery datasets"""
    data = []
    for project in projects:
        client = bigquery.Client(project=project)
        datasets = client.list_datasets()
 
        for dataset in datasets:
            dataset_id = dataset.dataset_id
            dataset_ref = client.get_dataset(dataset_id)
            tables = client.list_tables(dataset_id)
            for table in tables:
                full_table_id = table.full_table_id.replace(':', '.')
                table_ref = client.get_table(full_table_id)
                item = {'dataset_id': dataset_id,
                        'project_id': project,
                        'table_id': table_ref.table_id,
                        'dataset_description': dataset_ref.description,
                        'table_modified': table_ref.modified.strftime("%Y-%m-%d %H:%M:%S"),
                        'table_created': table_ref.created.strftime("%Y-%m-%d %H:%M:%S"),
                        'table_description': table_ref.description,
                        'table_num_bytes': table_ref.num_bytes,
                        'table_num_rows': table_ref.num_rows,
                        'table_partitioning_type': table_ref.partitioning_type,
                        'table_type': table_ref.table_type,
                        'table_columns' : field_details(table_ref),
                        
                        }
                data.append(item)
    return data

In [99]:
def field_details(table_ref):
    schema_dict = table_ref.to_api_repr()["schema"]["fields"]
    fields = [_["name"] for _ in schema_dict]
    return fields

In [100]:
res = get_bq_data()

In [101]:
import pandas as pd

In [102]:
df =pd.DataFrame(res)

In [103]:
df.head(2)

,dataset_description,dataset_id,project_id,table_columns,table_created,table_description,table_id,table_modified,table_num_bytes,table_num_rows,table_partitioning_type,table_type
0,None,austin_311,bigquery-public-data,"[unique_key, complaint_type, complaint_descrip...",2017-05-18 14:29:56,None,311_service_requests,2019-01-05 00:58:32,185335640,640525,None,TABLE
1,None,austin_bikeshare,bigquery-public-data,"[station_id, name, status, latitude, longitude...",2017-05-25 03:43:34,None,bikeshare_stations,2019-01-24 10:44:40,7384,94,None,TABLE


In [104]:
df.to_csv("/home/krishna/projects/lineage/data/public_dataset_metadata_fin.csv")